In [11]:
import geopandas as gpd
import subprocess

comercial_df = gpd.read_file('./data/LOTES/COMERCIAL/SHP/lotes.shp')
servicos_df = gpd.read_file('./data/LOTES/SERVIÇOS/SHP/lotes.shp')

In [12]:
def to_kml(df, output):
    command = "ogr2ogr -f KML {0}.kml {0}.json ".format(output)
    with open('{0}.json'.format(output), 'w') as f:
        f.write(df.to_crs(epsg=4326).to_json())
        
    subprocess.call(command,shell=True)
    
def export(df, output):
    kml_output = './OUTPUT/KML/' + output 
    csv_output = './OUTPUT/CSV/' + output + '.csv'
    shp_output = './OUTPUT/SHP/' + output
    
    to_kml(df, kml_output)
    df.to_file(shp_output)
    without_geometry = df.loc[:, df.columns != 'geometry']
    without_geometry.to_csv(csv_output)

In [13]:
def classify_comercio(row, column, dic):
    string = row[column]
    if string in dic['alimentacao']:
        return 1
    if string in dic['genAlimeticios']:
        return 2 
    if string in dic['artigosDoLar']:
        return 3 
    if string in dic['artigosDiversos']:
        return 4
    if string in dic['conveniencia']:
        return 5  
    if string in dic['vestuario']:
        return 6  
    if string in dic['construcao']:
        return 7
    if string in dic['automotivo']:
        return 8


def classify_servicos(row, column, dic):
    string = row[column]
    if string in dic['servicosAutomotivos']:
        return 8
    if string in dic['entretenimento']:
        return 9
    if string in dic['servicosDiversos']:
        return 10  
    if string in dic['servicosPessoais']:
        return 11
    if string in dic['servicosMedicos']:
        return 12
    
def classify_comercio_desc(row, column, dic):
    string = row[column]
    if string in dic['alimentacao']:
        return 'Alimentação'
    if string in dic['genAlimeticios']:
        return 'Gêneros Alimentícios'
    if string in dic['artigosDoLar']:
        return 'Artigos do Lar'
    if string in dic['artigosDiversos']:
        return 'Artigos Diversos'
    if string in dic['conveniencia']:
        return 'Conveniência'  
    if string in dic['vestuario']:
        return 'Vestuários'  
    if string in dic['construcao']:
        return 'Construção'
    if string in dic['automotivo']:
        return 'Automotivo'


def classify_servicos_desc(row, column, dic):
    string = row[column]
    if string in dic['servicosAutomotivos']:
        return 'Automotivo'
    if string in dic['entretenimento']:
        return 'Entretenimento'
    if string in dic['servicosDiversos']:
        return 'Serviços Diversos'  
    if string in dic['servicosPessoais']:
        return 'Serviços Pessoais'
    if string in dic['servicosMedicos']:
        return 'Serviços Médicos'

In [14]:
servicos_dict = {}
servicos_dict['servicosAutomotivos'] =  [ 'LOCACAO DE VEICULOS,EMBARCACOES OU AERONAVES',
'OFICINA MECANICAS OU LANTERNAGENS E PINTURA'
]

servicos_dict['servicosPessoais'] = [ 'HIGIENE E ESTETICA PESSOAL',
 'ESCOLAS DE ESPORTES OU LUTAS',
 'PREST.SERV-HIG.PES.E COND.FIS.',
 'ACADEMIA DE GINASTICA OU MUSCULACAO'
]

servicos_dict['entretenimento'] = [  'LOTERIAS,JOGOS DE APOSTAS OU BINGOS',
 'RECREACAO INFANTIL OU PARQUE DE DIVERSAO',
 'LOCACAO DE FILMES,DISCO,JOGOS OU CDS'

]

servicos_dict['servicosMedicos'] = [  'FISIOTERAPIA E RECUPERACAO',
 'PREST.SERV-SAUDE',
 'CONSULTORIO MEDICO',
 'CLINICA MEDICA',
 'CLINICA ODONTOLOGICA',
 'CLINICA DE PSICOLOGIA',
 'RADIOLOGIA',
 'FONOAUDIOLOGIA',
 'MEDICO',
]

servicos_dict['servicosDiversos'] = [  'AGENCIA DE TURISMO,VIAGENS OU VENDAS DE PASSAGENS',
'ESTUDIO FOTOGRAFICO',
 'CONSTRUCAO CIVIL E ATIVIDADES AFINS',
 'PREST.SERV-EDUCACAO',
 'ASSISTENCIA SOCIAL,RELIGIOSA OU BENEFICENTE',
 'ENSINO DE 1 GRAU,2 GRAU OU 3 GRAU',
 'SERVICOS DE TELECOMUNICACOES',
 'CORRETAGEM DE PLANOS DE SAUDE OU SEGUROS',
 'PREST.SERV-ARQ.CONST. CIVIL',
 'CORRETAGEM DE BENS MOVEIS',
 'INSTITUICOES FINANCEIRAS GERAL',
 'SERVICOS DE BUFFET',
 'PREST.SERV-FIN.SEC.E CAPITAL.',
 'LOCACAO DE MAQUINAS,APARELHOS OU EQUIPAMENTOS',
 'CURSOS DE IDIOMAS',
 'CORRETAGEM DE BENS IMOVEIS',
 'MATERNAL INFANTIL OU CRECHES',
 'AUTO-ESCOLA',
 'ENSINO TECNICO,INDUSTRIAL OU COMERCIAL',
 'COPIAS EM GERAL',
 'OFICINA DE FERREIRO - SERRALHERIA',
]

In [15]:
comercial_dict = {}
comercial_dict['genAlimeticios'] =  ['SUPERMERCADO,OU EMPORIO E MERCEARIAS',
'SUPERMERCADOS',
'PRODUTOS ALIMENTICIOS OU BEBIDAS',
'ALIMENTOS, BEBIDAS EM GERAL',
'FRIGORIFICOS, CARNES E AVES',
'SUPERMERCADO, OU EMPORIO E MERCEARIA SEM LIXO ORGANICO',
'ESTIVAS E CEREAIS']

comercial_dict['conveniencia'] = [ 'MEDICAMENTOS,COSMETICOS OU PERFUMES E CONGENERES',
'PAPELARIA,ARMARINHOS,BOUTIQUE OU PRESENTES E BOMBONIERE',
'MEDICAMENTOS,COSMETICOS OU PERFUMES',
'ARTIGOS DE TABACARIA',
'FLORICULTURA',
'CINE,FOTO OU SOM',
'PRODUTOS DE PAPELARIA',
'LIVROS JORNAIS OU REVISTAS',
'MAGAZINE',
'LOJAS DE DEPARTAMENTOS'
]

comercial_dict['vestuario'] = [ 'JOIAS,RELOGIOS,OTICA OU OBJETOS DE ARTE',
'TECIDOS,CONFECCOES,CALCADOS OU ESPORTES',
'CONFECCOES,CALCADOS BIJUTERIAS E CONGENERES',
'ARTIGOS DE COUROS E TANANTES'
]

comercial_dict['artigosDoLar'] = [ 'ELETRO ELETRONICOS OU INFORMATICA',
'MOVEIS EM GERAL',
'ARTIGOS DE ESPUMA E COLCHOES',
'ARTIGOS PARA DECORACAO,VIDROS E CONGENERES',
'PRODUTOS PLASTICOS E DESCARTAVEIS',
'ARTIGOS PLASTICOS E DESCARTAVEIS',
'ART. DE HIGIENE LIMPEZA EM GERAL',
'ANIMAIS E PRODUTOS VETERINARIOS'
]

comercial_dict['artigosDiversos'] = ['COMERCIO A VAREJO',
'ARTIGOS DE ARTESANATO',
'COMERCIAL -COMERCIO VAREJISTA',
'COMERCIO FECHADO',
'ARTIGOS MEDICOS,HOSPITALARES OU ODONTOLOGICO',
'MAQUINAS,  EQUIPAMENTOS E MATERIA PRIMA PARA PANIFICACAO',
'ARTIGOS FUNERARIOS',
'ARTIGOS AGROPECUARIOS',
'PRODUTOS QUIMICOS',
'MAQUINAS, APARELHOS E EQUIP. OU PECAS',
'ARTIGOS PARA CACA E PESCA E CONGEN  ERES',
'PRODUTOS ALIMENTICIOS PARA ANIMAIS',
'PRODUTOS MUSICAIS E INSTRUMENTOS',
'ARTIGOS RELIGIOSOS EM GERAL',
'EXPORTACAO E IMPORTACAO',
'PRODUTOS DE AGROPECUARIA',
'ARTIGOS PIROTECNICOS',
]

comercial_dict['construcao'] = ['MATERIAL DE CONSTRUCAO,MADEIRAS OU PEDRAS',
'EQUIPAMENTOS,APARELHOS,FERRAMENTAS OU PECAS',
'FERRAGENS,MATERIAIS ELETRICOS,HIDRAULICOS E CONGENERES',
'ADUBOS,TINTAS OU VERNIZES E CONGENERES',
'CASAS DE FERRAGENS'
]

comercial_dict['alimentacao'] = ['BAR, RESTAURANTE, LANCHONETE',
'LANCHONETE',
'SORVETERIA'
]

comercial_dict['automotivo'] = [ 'CONCESSIONARIAS DE VEICULOS E AUTO PECAS',
'COMBUSTIVEIS,LUBRIFICANTES OU GLP',
'VEICULOS E MAQUINAS PESADAS',
'PNEUS E ARTEFATOS DE BORRACHA',
'BICICLETAS, MOTOS E PECAS'
]

In [16]:
comercial_df['classificacao'] = comercial_df.apply(classify_comercio, axis=1, args=('Atividade', comercial_dict))
servicos_df['classificacao'] = servicos_df.apply(classify_servicos, axis=1, args=('Atividade', servicos_dict))
comercial_df['classificacao_desc'] = comercial_df.apply(classify_comercio_desc, axis=1, args=('Atividade', comercial_dict))
servicos_df['classificacao_desc'] = servicos_df.apply(classify_servicos_desc, axis=1, args=('Atividade', servicos_dict))

In [17]:
lotes_df = comercial_df.append(servicos_df, ignore_index=True)

In [18]:
recorte_maior = gpd.read_file('./data/BASES/limite_recorte/limite_recorte_maior.shp')
lotes_recorte =  gpd.overlay(lotes_df, recorte_maior, how='intersection')

In [23]:
bairros = gpd.read_file('./data/BASES/bairros/bairros.shp')
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
bairros_recorte = bairros.loc[bairros['NM_BAIRRO'].isin(bairros_interesse)]
bairros_recorte.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [26]:
lotes_df = lotes_df.to_crs('epsg:3857')

In [40]:
lotes_df

,Grupo,Atividade,Nume_Ativ,Segmento,Situacao,Area_Terre,Area_Total,Area_Predi,geometry,classificacao,classificacao_desc
0,COMERCIO VAREJISTA,COMERCIO A VAREJO,20133,COMERCIAL,ATIVO,386.75,51.60,51.60,"POLYGON ((-3878827.923 -788899.376, -3878807.5...",4.0,Artigos Diversos
1,COMERCIO VAREJISTA,"ALIMENTOS, BEBIDAS EM GERAL",20101,COMERCIAL,ATIVO,360.00,191.79,191.79,"POLYGON ((-3878822.951 -787820.017, -3878811.2...",2.0,Gêneros Alimentícios
2,COMERCIO VAREJISTA,ART. DE HIGIENE LIMPEZA EM GERAL,20135,COMERCIAL,ATIVO,360.00,119.00,119.00,"POLYGON ((-3878989.517 -787831.481, -3878959.9...",3.0,Artigos do Lar
3,COMERCIO VAREJISTA,"BAR, RESTAURANTE, LANCHONETE",20126,COMERCIAL,ATIVO,360.00,90.43,90.43,"POLYGON ((-3878976.989 -787878.599, -3878947.6...",1.0,Alimentação
4,COMERCIO VAREJISTA,"EQUIPAMENTOS,APARELHOS,FERRAMENTAS OU PECAS",20106,COMERCIAL,ATIVO,360.00,152.86,152.86,"POLYGON ((-3878819.372 -787976.056, -3878807.6...",7.0,Construção
...,...,...,...,...,...,...,...,...,...,...,...
10781,ADMINISTRATIVO GERENCIAL,ADM DE BENS OU NEGOCIOS .CONSORCIOS OU FUNDOS ...,10101,PRESTACAO DE SERVICOS,ATIVO,360.00,196.65,196.65,"POLYGON ((-3879105.236 -793805.216, -3879093.2...",NaN,None
10782,LOCACAO E GUARDA DE BENS E VALORES,VIGILANCIA OU SEGURANCA DE VALORES OU BENS,11411,PRESTACAO DE SERVICOS,ATIVO,3279.90,1581.18,584.42,"POLYGON ((-3878876.020 -793720.936, -3878846.8...",NaN,None
10783,EDUCACAO,MATERNAL INFANTIL OU CRECHES,10604,PRESTACAO DE SERVICOS,ATIVO,820.00,1130.27,1118.27,"POLYGON ((-3878819.811 -793616.690, -3878818.0...",10.0,Serviços Diversos
10784,INTERMEDIACAO E REPRESENTACAO,SERVICOS DE BUFFET,11312,PRESTACAO DE SERVICOS,ATIVO,4000.00,2463.88,2463.88,"POLYGON ((-3878789.673 -793901.826, -3878812.9...",10.0,Serviços Diversos


In [61]:
lotes_with_bairros = gpd.sjoin(lotes_df, bairros_recorte, how="inner", op='intersects')

grouped_lotes = lotes_with_bairros.groupby(['NM_BAIRRO', 'classificacao_desc']).agg({'classificacao_desc': 'count', 
                                                        'Area_Terre': 'sum',
                                                        'Area_Predi': 'sum',
                                                        'Area_Total': 'sum'
                                                       }).rename(columns={'classificacao_desc':'n_empreeendimentos'})

grouped_lotes.to_csv('./OUTPUT/CSV/lotes_por_bairro.csv')

In [62]:
grouped_lotes

n_empreeendimentos  Area_Terre  Area_Predi  \
NM_BAIRRO classificacao_desc                                                 
Estados   Alimentação                           22    13111.90     5904.75   
          Artigos Diversos                      70   191896.76    40674.46   
          Artigos do Lar                        19     9354.56     4539.10   
          Automotivo                            13    15810.35     8357.99   
          Construção                             9     7943.30     4122.01   
...                                            ...         ...         ...   
Torre     Gêneros Alimentícios                  58    24529.31    11508.80   
          Serviços Diversos                    137   116239.11    46571.92   
          Serviços Médicos                      47    21803.87     9354.93   
          Serviços Pessoais                     14     4260.33     1629.10   
          Vestuários                            25     7433.73     3099.99   

                                Area_Total  
NM_BAIRRO classificacao_desc                
Estados   Alimentação              6873.97  
          Artigos Diversos        56791.83  
          Artigos do Lar           6716.60  
          Automotivo               9243.47  
          Construção               4139.76  
...                                    ...  
Torre     Gêneros Alimentícios    13054.90  
          Serviços Diversos       52479.63  
          Serviços Médicos        12117.76  
          Serviços Pessoais        2450.59  
          Vestuários               4282.55  

[68 rows x 4 columns]

In [53]:
export(lotes_df, 'lotes_varejo')
export(lotes_recorte, 'lotes_varejo_recorte')

In [54]:
#EXPORT WITH DESCRIPTION AS CLASSIFICACAO, ONLY ROWS IN VAREJO

def export_classificacao(df, output):
    df['classificacao'] = df['classificacao_desc']
    df = df.drop('classificacao_desc', axis=1)
    df = df.dropna(axis=0, how='any', subset=['classificacao'])
    export(df, output)
    
export_classificacao(lotes_df, 'lotes_varejo')
export_classificacao(lotes_recorte, 'lotes_varejo_recorte')